In [ ]:
#wildfire
import folium
import pandas as pd

# Upload CSV
data = pd.read_csv('focos_mensal_br_202301 - SP.csv')  # Replace with the correct name of the CSV file

# Create map
wildfireSP_map = folium.Map(location=[-23.5, -46.5], zoom_start=6, max_zoom=10, min_zoom=4)

# Dataframe and markers
for index, row in data.iterrows():
    popup_info = f"""
    Municipality: {row['municipio']}<br>
    State: {row['estado']}<br>
    Date/Time: {row['data_hora_gmt']}<br>
    Days without rain: {row['numero_dias_sem_chuva']}<br>
    Precipitation: {row['precipitacao']}<br>
    Fire Risk: {row['risco_fogo']}<br>
    Biome: {row['bioma']}<br>
    FRP: {row['frp']}
    """

    folium.Marker(
        location=[row['lat'], row['lon']],  # Latitude and longitude columns from the CSV
        popup=popup_info,  # Detailed information in the popup
        icon=folium.Icon(color='red')  # Red marker
    ).add_to(wildfireSP_map)

# Show map
wildfireSP_map

# Save map in HTML file
wildfireSP_map.save('wildfireSP_map.html')  # Save the map to an HTML file


In [ ]:
#precipitation
import folium
from folium.plugins import HeatMap
import pandas as pd
from branca.colormap import LinearColormap

# Load precipitation data
data_precipitation = pd.read_csv('focos_mensal_br_202301 - RJ.csv')

# Clean data by removing rows with NaN values in relevant columns
data_precipitation = data_precipitation[['lat', 'lon', 'precipitacao']].dropna()

# Create a centered map
precipitation_map = folium.Map(location=[-23.5, -46.5], zoom_start=6, max_zoom=10, min_zoom=4)

# Prepare data for the HeatMap
heat_data = [[row['lat'], row['lon'], row['precipitacao']] for index, row in data_precipitation.iterrows()]

# Create a colormap for the legend
colormap = LinearColormap(colors=['lightblue', 'blue', 'darkblue'],
                          vmin=data_precipitation['precipitacao'].min(),
                          vmax=data_precipitation['precipitacao'].max())

# Add the HeatMap to the map
HeatMap(heat_data, radius=30, gradient={0: 'lightblue', 0.5: 'blue', 1: 'darkblue'}).add_to(precipitation_map)

# Add precipitation markers to the map
for index, row in data_precipitation.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f'Precipitation: {row["precipitacao"]} mm',
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(precipitation_map)

# Create HTML for the legend
colormap_html = colormap._repr_html_()

# Add the legend to the map
precipitation_map.get_root().html.add_child(folium.Element("""
    <style>
        .colormap {
            position: absolute;
            top: 10px;
            left: 70px;
            z-index: 1000;
        }
    </style>
    <div class='colormap'>""" + colormap_html + """</div>
"""))

# Save the map to an HTML file
precipitation_map.save('precipitation_mapRJ.html')


In [ ]:
import folium
from folium.plugins import HeatMap
import pandas as pd
from branca.colormap import LinearColormap

# Load precipitation data
data_precipitation = pd.read_csv('focos_mensal_br_202301 - SP.csv')

# Clean the data by removing rows with NaN values in relevant columns
data_precipitation = data_precipitation[['lat', 'lon', 'risco_fogo']].dropna()

# Define a fire risk threshold (adjust as necessary)
risk_threshold = data_precipitation['risco_fogo'].quantile(0.75)  # Example: 75% of the data

# Filter the data to include only locations with fire risk above the threshold
high_risk_data = data_precipitation[data_precipitation['risco_fogo'] >= risk_threshold]

# Create a centered map
fire_risk_map = folium.Map(location=[-23.5, -46.5], zoom_start=6, max_zoom=10, min_zoom=4,
                            scrollWheelZoom=True, dragging=True, zoom_control=True)

# Prepare the data for the HeatMap
heat_data = [[row['lat'], row['lon'], row['risco_fogo']] for index, row in high_risk_data.iterrows()]

# Create a colormap for fire risk (adjust colors as necessary)
colormap = LinearColormap(colors=['green', 'yellow', 'orange', 'red'],
                          vmin=data_precipitation['risco_fogo'].min(),
                          vmax=data_precipitation['risco_fogo'].max())

# Add the HeatMap to the map
HeatMap(heat_data, radius=30, gradient={0: 'green', 0.5: 'orange', 1: 'red'}).add_to(fire_risk_map)

# Add fire risk markers to the map
for index, row in high_risk_data.iterrows():
    folium.Marker(
        location=[row['lat'], row['lon']],
        popup=f'Fire Risk: {row["risco_fogo"]}',
        icon=folium.Icon(color='orange', icon='fire')
    ).add_to(fire_risk_map)

# Create a div to position the legend in the top left corner
fire_risk_map.get_root().html.add_child(folium.Element("""
    <style>
        .colormap {
            position: absolute;
            top: 10px;
            left: 70px;  /* Adjust the legend position here */
            z-index: 1000;
        }
    </style>
    <div class='colormap'>""" + colormap._repr_html_() + """</div>
"""))

# Save the map to an HTML file
fire_risk_map.save('high_fire_risk_map_SP.html')

# Display the map


In [ ]:
import pandas as pd
import folium

# Assuming you have loaded a file named 'sdei-global-uhi-2013 - Sheet1.csv'
df = pd.read_csv('sdei-global-uhi-2013 - Sheet1.csv')

# Create a map
uhi_map = folium.Map(location=[-23.0, -46.0], zoom_start=6)

# Add markers
for _, row in df.iterrows():
    # Create the content for the pop-up
    popup_content = (
        f"<strong>NASA UHI Data</strong><br>"
        f"<strong>Location:</strong> {row['NAME']}<br>"
        f"<strong>Average Daytime Temperature:</strong> {row['URB_D_MEAN']} °C<br>"
        f"<strong>Buffer Average Daytime Temperature:</strong> {row['BUF_D_MEAN']} °C<br>"
        f"<strong>Estimated Population in 2000:</strong> {row['ES00POP']}<br>"
        f"<strong>Daytime Temperature Difference:</strong> {row['D_T_DIFF']} °C<br><br>"
        f"<a href='#' onclick='alert(`Definitions of terms used:`);'>Click for Definitions</a>"
    )

    # Add the marker with the pop-up
    folium.Marker(
        location=[row['LATITUDE'], row['LONGITUDE']],
        popup=folium.Popup(popup_content, max_width=300),  # Set the maximum width of the pop-up
        icon=folium.Icon(color='blue')
    ).add_to(uhi_map)

# Display the map
uhi_map.save('sdei-global-uhi.html')
